In [1]:
#运行这个代码可以让Python不显示warnings
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import geopandas
import geopandas as gpd
import matplotlib.pyplot as plt

In [2]:
xianlu=1088

In [3]:
#沿线300米栅格

In [4]:
#运行这个代码可以让Python不显示warnings
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import geopandas
import geopandas as gpd
import pyproj
xinling = pd.read_csv(r'F:\\data\\扬州.csv',header=None,encoding='utf-8')#这样读取默认第一行是表头\n",
xinling['区编号']=xinling.iloc[:,0]
xinling['经度']=xinling.iloc[:,1]
xinling['纬度']=xinling.iloc[:,2]
QSY=xinling.loc[:,['区编号','经度','纬度']]
line = pd.read_csv(r'F:\\data\\扬州公交\\line_5_6\\线路shp\\'+str(xianlu)+'_line.csv',encoding='gbk')#默认第一行是表头
#-*-coding=utf-8 -*-\n",
import pyproj
import pandas as pd
import numpy as np
#data = pd.read_csv(\"test.csv\",sep=',',header='infer')#将第一行作为表头,
p1 = pyproj.Proj(init="epsg:4326")#wgs坐标系统的EPSG Code,
p2 = pyproj.Proj(init="epsg:32651")#UTM 51N坐标系统的EPSG Code\n",
lonprj2, latprj2 = pyproj.transform(p1, p2,line['LONGITUDE'], line['LATITUDE'])
lonprj3, latprj3 = pyproj.transform(p1, p2,QSY['经度'], QSY['纬度'])
##将数组转换为PD\n",
line['lons']=pd.DataFrame(lonprj2)
line['lats']=pd.DataFrame(latprj2)
QSY['lons']=pd.DataFrame(lonprj3)
QSY['lats']=pd.DataFrame(latprj3)
from shapely.geometry import Point
#对栅格GPS点生成点要素，并转换为一个GeoDataFrame，方便后续的地图匹配\n",
lon=QSY['lons']
lat=QSY['lats']
list=[]
for i in range(0,len(QSY)):
    g=Point(lon[i],lat[i])
    list.append(g)
    pass
list = gpd.GeoDataFrame(list)###转换为pd形式
QSY['geometry']=list
QSY['geometry']
#生成线路的线要素
from shapely.geometry import LineString
dianlon=line['lons']
dianlat=line['lats']
dianlist=[]
for i in range(0,len(line)):
    g=(dianlon[i],dianlat[i])
    dianlist.append(g)
    pass
dianlist
linestr = LineString(dianlist)
line['geometry']=linestr
line['geometry']
#地图匹配
lineshp=line['geometry'].iloc[1]
QSY['project'] = QSY['geometry'].apply(lambda r:lineshp.project(r))
#原始的坐标点存储在一个字段中
QSY['geometry_orgin'] = QSY['geometry']
#生成匹配的坐标点
QSY['geometry'] = QSY['project'].apply(lambda r:lineshp.interpolate(r))###差值拟合匹配
#批量计算原始点和匹配点之间的距离
QSY['diff'] = QSY.apply(lambda r:r['geometry_orgin'].distance(r['geometry']),axis = 1)
#储存
QSY.to_csv(r'F:\data\扬州公交\手机信令梳理\5月6号\\'+'NEW'+str(xianlu)+'\\QSY.csv',index=None)
#stop.to_csv(r'F:\data\扬州公交\手机信令梳理\5月6号\1001\stop.csv',index=None)
line.to_csv(r'F:\data\扬州公交\手机信令梳理\5月6号\\'+'NEW'+str(xianlu)+'\\line.csv',index=None)
QSY = pd.read_csv(r'F:\data\扬州公交\手机信令梳理\5月6号\\'+'NEW'+str(xianlu)+'\\QSY.csv',encoding='utf-8')#默认第一行是表头，所有栅格地图匹配后
lineQSY=QSY[QSY['diff']<300]##先将全市范围缩减到沿线300米以内，在此基础上，匹配栅格的最近站点
lineQSY.to_csv(r'F:\data\扬州公交\手机信令梳理\5月6号\\'+'NEW'+str(xianlu)+'\lineQSY300.csv',index=None)

In [5]:
##信令匹配

In [6]:
import os
import re
import datetime
import numpy as np
import pandas as pd
#运行这个代码可以让Python不显示warnings
import warnings
warnings.filterwarnings("ignore")
import geopandas
import geopandas as gpd
import pyproj
#-*-coding=utf-8 -*-\n",
import pyproj

In [7]:
def convert(n):
    return str(datetime.timedelta(seconds = n))
def t2s(t):
    h,m,s = t.strip().split(":")
    return int(h) * 3600 + int(m) * 60 + int(s)

In [8]:
PRODUCTID = pd.read_csv(r'F:\data\扬州公交\手机信令梳理\5月6号\\'+'NEW'+str(xianlu)+'\\PRODUCTID.csv',encoding='gbk')
for checi in range(0,len(PRODUCTID)):#合计22辆车有数据34
#for checi in range(2,35):#合计22辆车有数据34
#for checi in range(1,2):#合计22辆车有数据34
    ###统计文件夹内子文件数量
    dir_count = 0
    file_count = 0
    for root, dirs, filenames in os.walk(r'F:\data\扬州公交\手机信令梳理\5月6号\\'+'NEW'+str(xianlu)+'\\'+str(checi)):
        for dir in dirs:
            dir_count += 1
        for file in filenames:
            file_count += 1
    print ('dir_count ', dir_count)
    print ('file_count ', file_count)
    if file_count>4:                                
        TIME=pd.read_csv(r'F:\data\扬州公交\手机信令梳理\5月6号\\'+'NEW'+str(xianlu)+'\\'+str(checi)+'\\'+'TIME.csv')
        TIME['提前十五分钟时间']=None
        TIME['延迟十五分钟时间']=None
        print('checi ',checi)
        for i in range(0,len(TIME)):#!!!!!!
            t1=int(TIME['出发时间'].iloc[i])-900
            TIME['提前十五分钟时间'].iloc[i]=convert(t1)
            t2=int(TIME['结束时间'].iloc[i])+900
            TIME['延迟十五分钟时间'].iloc[i]=convert(t2)
            import datetime####时间记得更新，前后拓展15分钟
            tangci=i+1#!!!!!!
            d_time = datetime.datetime.strptime(TIME['提前十五分钟时间'].iloc[i], '%H:%M:%S')##!!!!
            d_time1 = datetime.datetime.strptime(TIME['延迟十五分钟时间'].iloc[i], '%H:%M:%S')###!!!!注意区分白天和晚班车
            ####！！！！修改日期 ,不能提前
            arrive=pd.read_csv(r'F:\data\扬州公交\yangzhou_20210501-17_res\yangzhou_res\arriveRes\20210506.csv',header=None,names=['日期','到达时间','D编号','到达人数','出发时间','O编号','出发人数'],encoding='utf-8')
            ###带有上下行属性的站点信息
            stop=pd.read_excel(r'F:\data\扬州公交\上下行站点\\'+str(xianlu)+'.xlsx')
            ####读取沿线300米的栅格数据
            lineQSY= pd.read_csv(r'F:\data\扬州公交\手机信令梳理\5月6号\\'+'NEW'+str(xianlu)+'\\'+'lineQSY300.csv',encoding='utf-8')#默认第一行是表头，所有栅格地图匹配后
            line = pd.read_csv(r'F:\data\扬州公交\手机信令梳理\5月6号\\'+'NEW'+str(xianlu)+'\\'+str(checi)+'\\'+str(tangci)+'_line.csv',encoding='utf-8')
            ###区分上下行
            flag=TIME['上1下0'].iloc[i]
            stop=stop[(stop['站点属性']==flag)|(stop['站点属性']==3)]
            #print(stop)
            ###切分时间字段
            ###筛选出这段时间内到达的栅格人群，线路
            d=arrive['到达时间'].str.split('-',expand=True)
            d['起始时间']=d.loc[:,[0]]
            d['结束时间']=d.loc[:,[1]]
            arrive['到达起始时间']=d['起始时间']
            arrive['到达结束时间']=d['结束时间']
            o=arrive['出发时间'].str.split('-',expand=True)
            o['起始时间']=o.loc[:,[0]]
            o['结束时间']=o.loc[:,[1]]
            arrive['出发起始时间']=o['起始时间']
            arrive['出发结束时间']=o['结束时间']
            #根据时间筛选栅格数据
            list=[] 
            n=len(arrive)###!!!!!这个参数记得改\n",
            for i in range(0, n):    
                # 当前时间\n",
                s_time = datetime.datetime.strptime(arrive.loc[i,'出发起始时间'], '%H:%M')
                e_time = datetime.datetime.strptime(arrive.loc[i,'到达结束时间'], '%H:%M')
                # 当前时间\n",
                #n_time = datetime.datetime.now()\n","    #判断时间\n","    #n_time =BUS_GPS.loc[1,'RECDATETIME']\n",
                # 判断当前时间是否在范围时间内\n",
                if s_time > d_time and e_time<d_time1:
                    k=i
                    list.append(k)##要用圆括号,且不要写成list=list.append(k)的赋值形式\n",
                    #读取经纬度
                    pass
            DTSY=arrive.loc[list,:]
            print('时间划分完毕')
            arrive=DTSY
            ##根据沿线300的栅格数据，筛选沿线300的手机信令数据
            df1 = pd.DataFrame()###新建一个pandas\n",
            for i in range(0,len(lineQSY)):
                sg=arrive.loc[arrive['O编号'].str.contains(lineQSY['区编号'].iloc[i])] 
                df1 = df1.append(sg,ignore_index = True)
            #df1.drop_duplicates(subset=['O编号'],keep='first',inplace=True)#！！！不能去重
            #筛选到达点在线路上的od对
            df2 = pd.DataFrame()###新建一个pandas\n",
            for i in range(0,len(lineQSY)):
                sg=df1.loc[df1['D编号'].str.contains(lineQSY['区编号'].iloc[i])] 
                df2 = df2.append(sg,ignore_index = True)
            #df2.drop_duplicates(subset=['D编号'],keep='first',inplace=True)#！！！不能去重
            print('空间划分完毕')
            QSY300=df2
            QSY300.to_csv(r'F:\data\扬州公交\手机信令梳理\5月6号\\'+'NEW'+str(xianlu)+'\\'+str(checi)+'\\'+str(tangci)+'_QSY300.csv',index=None)###全天5路沿线300米
            #TIME= pd.read_csv(r'F:\data\扬州公交\手机信令梳理\5月6号\\'+str(xianlu)+'\\'+str(checi)+'\\'+str(tangci)+'_line.csv',encoding='utf-8')#默认第一行是表头，所有栅格地图匹配后
            ##根据轨迹，给站点赋予时间戳
            #stop=pd.read_csv(r'F:\data\扬州公交\stop_5_6\1005.csv',encoding='utf-8')###导入地图匹配之后的站点信息
            #data = pd.read_csv(\"test.csv\",sep=',',header='infer')#将第一行作为表头,
            stop.reset_index(drop=True,inplace=True)#重新调整索引
            p1 = pyproj.Proj(init="epsg:4326")#wgs坐标系统的EPSG Code,
            p2 = pyproj.Proj(init="epsg:32651")#UTM 51N坐标系统的EPSG Code\n",
            #lonprj, latprj = pyproj.transform(p1, p2,line['LONGITUDE'], line['LATITUDE'])
            #line['lons']=pd.DataFrame(lonprj)
            #line['lats']=pd.DataFrame(latprj)
            lonprj1, latprj1 = pyproj.transform(p1, p2,stop['LONGITUDE'], stop['LATITUDE'])
            stop['lons']=pd.DataFrame(lonprj1)
            stop['lats']=pd.DataFrame(latprj1)
            from shapely.geometry import Point
            ##生成轨迹点的点要素
            lon=line['lons']
            lat=line['lats']
            stoplist=[]
            for i in range(0,len(line)):
                g=Point(lon[i],lat[i])
                stoplist.append(g)
                pass
            stoplist = gpd.GeoDataFrame(stoplist)###转换为pd形式
            list
            #line['geometry']=stoplist
            #line['geometry']
            import math
            ###将所有小区匹配到最近的站点上
            match=[]
            nearestd=[]
            r=stop.index
            #print('r')
            #print(r)
            for k in range(0,len(lineQSY)):###range写法要求索引连续完整
                distance=[]
                x2 = lineQSY.loc[k,'lons']
                y2 = lineQSY.loc[k,'lats']
                for i in r:        
                    x1 = stop.loc[i,'lons']
                    y1 = stop.loc[i,'lats']
                    distance.append(math.sqrt((x2-x1)**2+(y2-y1)**2))
                    #print(distance)
                # 求列表最小值及索引
                min_value = min(distance) # 求列表最小值
                #print(min_value)
                min_idx = distance.index(min_value) # 求最小值对应索引  
                nearestd.append(min_value)
                idx=r[min_idx]###这里，以及下一块，一定要注意，索引不连续的问题
                match.append(idx)
                #print('match')
                #print(match)
            ###最近站点整理
            lineQSY['最近站名索引']=match
            lineQSY['最近站距离']=nearestd
            lineQSY['最近站名']=None
            lineQSY['最近站编号']=None
            for k in range(0,len(lineQSY)):
                sy=lineQSY['最近站名索引'].loc[k]
                lineQSY['最近站名'].loc[k]=stop['STATIONNAME'].loc[sy]
                lineQSY['最近站编号'].loc[k]=stop['STATIONNO'].loc[sy]
            lineQSY.to_csv(r'F:\data\扬州公交\手机信令梳理\5月6号\\'+'NEW'+str(xianlu)+'\\STATIONQSY300.csv',index=None)
            if flag==1:
                print('栅格与上行站点匹配完毕')
            if flag==0:
                print('栅格与下行站点匹配完毕')
            ###将所有站点匹配到最近的轨迹点上
            #for k in range(0,len(lineQSY)):
            match=[]
            nearestd=[]
            r=stop.index
            r1=line.index
           # print(r1)
            for k in r:###range写法要求索引连续完整
                distance=[]
                x2 = stop.loc[k,'lons']
                y2 = stop.loc[k,'lats']
                for i in r1:        
                    x1 = line.loc[i,'lons']
                    y1 = line.loc[i,'lats']
                    #print(math.sqrt((x2-x1)**2+(y2-y1)**2))
                    distance.append(math.sqrt((x2-x1)**2+(y2-y1)**2))
                #print(distance)
                #print(len(distance))
                # 求列表最小值及索引
                min_value = min(distance) # 求列表最小值
                min_idx = distance.index(min_value) # 求最小值对应索引
                idx=r1[min_idx]
                nearestd.append(min_value)
                match.append(idx)
                #print('match')
                #print(match)
            ###最近轨迹点整理
            stop['最近轨迹点索引']=match
            stop['最近点距离']=nearestd
            stop['RECDATETIME']=0
            for k in r:
                sy=stop['最近轨迹点索引'].loc[k]
                stop['RECDATETIME'].loc[k]=line['RECDATETIME'].loc[sy]
            print('站点最近轨迹点匹配完毕')
            ##将栅格关联到站点
            spaceod=QSY300
            spaceod['出发站']=None
            spaceod['到达站']=None
            spaceod['出发站编号']=0
            spaceod['到达站编号']=0
            for i in range(0,len(lineQSY)):
                sg=spaceod.loc[spaceod['O编号'].str.contains(lineQSY['区编号'].iloc[i])] 
                #print(sg)
                k=sg.index
                spaceod['出发站'].loc[k]=lineQSY['最近站名'].iloc[i]
                spaceod['出发站编号'].loc[k]=lineQSY['最近站编号'].iloc[i]
            for i in range(0,len(lineQSY)):
                sg=spaceod.loc[spaceod['D编号'].str.contains(lineQSY['区编号'].iloc[i])] 
                #print(sg)
                k=sg.index
                spaceod['到达站'].loc[k]=lineQSY['最近站名'].iloc[i]
                spaceod['到达站编号'].loc[k]=lineQSY['最近站编号'].iloc[i]
            #获取站点的时间戳
            stod=spaceod
            #去除空值
            stod.dropna()
            stod['出发站车辆到站时间']=0
            stod['到达站车辆到站时间']=0
            for i in range(0,len(stod)):
                for j in r:
                    if stod['出发站编号'].iloc[i]==stop['STATIONNO'].loc[j]:
                        stod['出发站车辆到站时间'].iloc[i]=stop['RECDATETIME'].loc[j]
                    if stod['到达站编号'].iloc[i]==stop['STATIONNO'].loc[j]:
                            stod['到达站车辆到站时间'].iloc[i]=stop['RECDATETIME'].loc[j]       
            print('时间戳获取完毕')
            #去除原地不动的点
            list=[]
            r=stod.index
            for i in r:
                if stod['出发站车辆到站时间'].loc[i]==stod['到达站车辆到站时间'].loc[i]:
                    list.append(i)
            stod=stod.drop(list,axis=0)
            #统一时间格式                
            stod ['出发站车辆到站时间'] = stod ['出发站车辆到站时间'].map(lambda x: str(x)[9:])
            stod ['到达站车辆到站时间'] = stod ['到达站车辆到站时间'].map(lambda x: str(x)[9:])
            #print(stod)
            #转换时间格式，以便下一步计算
            #在这之前需要排个序,##以空格划分日期和时间
            #标准时间的换算，相对00：00：00过了多少秒
            stod['OST'] = stod['出发站车辆到站时间'].apply(lambda r:r.split(':')[0]).astype('int')*3600+\
            stod['出发站车辆到站时间'].apply(lambda r:r.split(':')[1]).astype('int')*60+\
            stod['出发站车辆到站时间'].apply(lambda r:r.split(':')[2]).astype('int')-900

            stod['OT'] = stod['出发站车辆到站时间'].apply(lambda r:r.split(':')[0]).astype('int')*3600+\
            stod['出发站车辆到站时间'].apply(lambda r:r.split(':')[1]).astype('int')*60+\
            stod['出发站车辆到站时间'].apply(lambda r:r.split(':')[2]).astype('int')
            stod['DT'] = stod['到达站车辆到站时间'].apply(lambda r:r.split(':')[0]).astype('int')*3600+\
            stod['到达站车辆到站时间'].apply(lambda r:r.split(':')[1]).astype('int')*60+\
            stod['到达站车辆到站时间'].apply(lambda r:r.split(':')[2]).astype('int')

            stod['DST'] = stod['到达站车辆到站时间'].apply(lambda r:r.split(':')[0]).astype('int')*3600+\
            stod['到达站车辆到站时间'].apply(lambda r:r.split(':')[1]).astype('int')*60+\
            stod['到达站车辆到站时间'].apply(lambda r:r.split(':')[2]).astype('int')+900

            stod['o1'] = stod['出发起始时间'].apply(lambda r:r.split(':')[0]).astype('int')*3600+\
            stod['出发起始时间'].apply(lambda r:r.split(':')[1]).astype('int')*60
            stod['o2'] = stod['出发结束时间'].apply(lambda r:r.split(':')[0]).astype('int')*3600+\
            stod['出发结束时间'].apply(lambda r:r.split(':')[1]).astype('int')*60

            stod['d1'] = stod['到达起始时间'].apply(lambda r:r.split(':')[0]).astype('int')*3600+\
            stod['到达起始时间'].apply(lambda r:r.split(':')[1]).astype('int')*60

            stod['d2'] = stod['到达结束时间'].apply(lambda r:r.split(':')[0]).astype('int')*3600+\
            stod['到达结束时间'].apply(lambda r:r.split(':')[1]).astype('int')*60
            #stod['OST'] = stod['出发站车辆到站时间'].apply(lambda r:r.split(' ')[1].split(':')[0]).astype('int')*3600+\
            #stod['出发站车辆到站时间'].apply(lambda r:r.split(' ')[1].split(':')[1]).astype('int')*60+\
            #stod['出发站车辆到站时间'].apply(lambda r:r.split(' ')[1].split(':')[2]).astype('int')-900
            ##以上为计算从零点到当前时刻的秒数
            #stod.sort_values(by='time_st', inplace=True, ascending=True)
            ##按照时间戳排序
            list=[]
            r=stod.index
            for i in r:
                if stod['OT'].loc[i]>stod['DT'].loc[i]:
                    list.append(i)
            stod=stod.drop(list,axis=0)
            ###删除路程十分钟以内
            list=[]
            r=stod.index
            for i in r:
                if stod['OT'].loc[i]>stod['DT'].loc[i]-600:
                    list.append(i)
            stod=stod.drop(list,axis=0)
            stod.loc[:,['出发时间','出发站车辆到站时间','出发人数','到达时间','到达站车辆到站时间','到达人数']]
            stod.to_csv(r'F:\data\扬州公交\手机信令梳理\5月6号\\'+'NEW'+str(xianlu)+'\\'+str(checi)+'\\'+'15min过滤之前'+str(tangci)+'.csv')
            stod2=stod
            list=[]
            r=stod2.index
            for i in r:
                if stod2['OST'].loc[i]<stod2['o2'].loc[i] and stod2['DST'].loc[i]>stod2['d1'].loc[i]:
                #if stod['OST'].loc[i]>stod['o2'].loc[i] and stod['OT'].loc[i]<stod['o1'].loc[i] and stod['DT'].loc[i]<stod['d2'].loc[i] and stod['DST'].loc[i]>stod['d1'].loc[i]:     
                    list.append(i)
            list
            index=list
            stod2.loc[index,['出发时间','出发站车辆到站时间','出发人数','到达时间','到达站车辆到站时间','到达人数']]
            stod2=stod2.loc[index,:]
            path=r'F:\data\扬州公交\手机信令梳理\5月6号\\'+'NEW'+str(xianlu)+'\\'+str(checi)+'\\'+'OD'+str(tangci)+'.csv'
            stod2.to_csv(path)
            data = pd.read_csv(path)
            data2=data.loc[:,['O编号','出发站','出发时间','D编号','到达站','到达时间','出发站编号','到达站编号','出发人数','到达人数','出发站车辆到站时间','到达站车辆到站时间']]
            #morning_data2 .drop_duplicates(subset=['出发站编号','到达站编号'], keep='first', inplace=True)
            #data2.to_excel(r'F:\data\扬州公交\手机信令梳理\5月3号\5号线\车次\0号车趟次\流量3.xlsx')
            data2.to_excel(r'F:\data\扬州公交\手机信令梳理\5月6号\\'+'NEW'+str(xianlu)+'\\'+str(checi)+'\\'+'流量'+str(tangci)+'.xlsx')
            print('OD计算完毕')
            TONGJI=stod2
            TONGJI.loc['合计'] =TONGJI.apply(lambda x:x.sum())
            print(TONGJI.loc['合计'])

dir_count  0
file_count  11
checi  0
时间划分完毕
空间划分完毕
栅格与下行站点匹配完毕
站点最近轨迹点匹配完毕
时间戳获取完毕
OD计算完毕
日期           0.0
到达时间         0.0
D编号          0.0
到达人数         0.0
出发时间         0.0
O编号          0.0
出发人数         0.0
到达起始时间       0.0
到达结束时间       0.0
出发起始时间       0.0
出发结束时间       0.0
出发站          0.0
到达站          0.0
出发站编号        0.0
到达站编号        0.0
出发站车辆到站时间    0.0
到达站车辆到站时间    0.0
OST          0.0
OT           0.0
DT           0.0
DST          0.0
o1           0.0
o2           0.0
d1           0.0
d2           0.0
Name: 合计, dtype: float64
时间划分完毕
空间划分完毕
栅格与上行站点匹配完毕
站点最近轨迹点匹配完毕
时间戳获取完毕
OD计算完毕
日期                                                  3031575900
到达时间         08:00-08:0508:00-08:0507:25-07:3007:40-07:4507...
D编号          wtubm1swtubm1swtubm1swtubm9wwtubm9wwtubm9wwtub...
到达人数                                                      3039
出发时间         07:45-07:5007:40-07:4507:20-07:2507:30-07:3507...
O编号          wtubq60wtubmchwtubm9qwtubqdnwtubqdnwtubq6hwtub...
出发人数                         

时间划分完毕
空间划分完毕
栅格与下行站点匹配完毕
站点最近轨迹点匹配完毕
时间戳获取完毕
OD计算完毕
日期                                                  2789049828
到达时间         15:45-15:5015:40-15:4515:55-16:0015:55-16:0016...
D编号          wtubm9wwtubm9wwtubqdnwtubqdnwtubqdnwtubm1nwtv0...
到达人数                                                      2136
出发时间         15:05-15:1015:35-15:4015:00-15:0515:00-15:0515...
O编号          wtubm1rwtubm16wtubk8ewtub7b4wtub78nwtubk27wtub...
出发人数                                                       138
到达起始时间       15:4515:4015:5515:5516:0015:3016:1516:1015:251...
到达结束时间       15:5015:4516:0016:0016:0515:3516:2016:1515:301...
出发起始时间       15:0515:3515:0015:0015:0015:2015:4515:3015:201...
出发结束时间       15:1015:4015:0515:0515:0515:2515:5015:3515:251...
出发站          市政府廉政广场来鹤台广场扬州火车站扬州火车站双博馆琼花观（银河电子城）琼花观（银河电子城）市...
到达站          珍园（萃园）珍园（萃园）文昌大桥文昌大桥文昌大桥市政府文昌东路曙光路文昌东路曙光路琼花观（银...
出发站编号                                                 14163366
到达站编号                                                 14280536
出发

时间划分完毕
空间划分完毕
栅格与下行站点匹配完毕
站点最近轨迹点匹配完毕
时间戳获取完毕
OD计算完毕
日期                                                  2061471612
到达时间         11:55-12:0010:40-10:4511:05-11:1010:40-10:4511...
D编号          wtv03r6wtubm9wwtubqdnwtubmcmwtv03hkwtubm9zwtub...
到达人数                                                      1586
出发时间         11:45-11:5010:35-10:4011:00-11:0510:35-10:4011...
O编号          wtv02uqwtubm1zwtubq13wtubm35wtv0250wtubm1zwtub...
出发人数                                                       102
到达起始时间       11:5510:4011:0510:4011:4011:0010:5511:1011:001...
到达结束时间       12:0010:4511:1010:4511:4511:0511:0011:1511:051...
出发起始时间       11:4510:3511:0010:3511:2510:5010:3511:0010:401...
出发结束时间       11:5010:4011:0510:4011:3010:5510:4011:0510:451...
出发站          佳源广场市政府琼花观（银河电子城）人民大厦（中医院）文昌东路曙光路市政府东方医院（中国移动）...
到达站          江都汽车客运站珍园（萃园）文昌大桥琼花观（银河电子城）京江大酒店（江都人民法院）珍园（萃园）...
出发站编号                                                 10516839
到达站编号                                                 10621637
出发

时间划分完毕
空间划分完毕
栅格与上行站点匹配完毕
站点最近轨迹点匹配完毕
时间戳获取完毕
OD计算完毕
日期                                                  6386519896
到达时间         14:10-14:1517:30-17:3517:50-17:5515:20-15:2513...
D编号          wtubm9wwtv03hkwtv03hkwtv03hkwtv03hkwtv03hkwtv0...
到达人数                                                      5316
出发时间         12:50-12:5516:05-16:1015:50-15:5514:00-14:0513...
O编号          wtv03r7wtv03r6wtv03r6wtv03r6wtv03r6wtv03r7wtv0...
出发人数                                                       318
到达起始时间       14:1017:3017:5015:2013:5017:2016:4517:1516:401...
到达结束时间       14:1517:3517:5515:2513:5517:2516:5017:2016:451...
出发起始时间       12:5016:0515:5014:0013:4016:0516:2017:1014:451...
出发结束时间       12:5516:1015:5514:0513:4516:1016:2517:1514:501...
出发站          江都汽车客运站江都汽车客运站江都汽车客运站江都汽车客运站江都汽车客运站江都汽车客运站江都汽车...
到达站          珍园（萃园）京江大酒店（江都人民法院）京江大酒店（江都人民法院）京江大酒店（江都人民法院）京...
出发站编号                                                 31666992
到达站编号                                                 35733331
出发

时间划分完毕
空间划分完毕
栅格与上行站点匹配完毕
站点最近轨迹点匹配完毕
时间戳获取完毕
OD计算完毕
日期           0.0
到达时间         0.0
D编号          0.0
到达人数         0.0
出发时间         0.0
O编号          0.0
出发人数         0.0
到达起始时间       0.0
到达结束时间       0.0
出发起始时间       0.0
出发结束时间       0.0
出发站          0.0
到达站          0.0
出发站编号        0.0
到达站编号        0.0
出发站车辆到站时间    0.0
到达站车辆到站时间    0.0
OST          0.0
OT           0.0
DT           0.0
DST          0.0
o1           0.0
o2           0.0
d1           0.0
d2           0.0
Name: 合计, dtype: float64
dir_count  0
file_count  11
checi  4
时间划分完毕
空间划分完毕
栅格与下行站点匹配完毕
站点最近轨迹点匹配完毕
时间戳获取完毕
OD计算完毕
日期           0.0
到达时间         0.0
D编号          0.0
到达人数         0.0
出发时间         0.0
O编号          0.0
出发人数         0.0
到达起始时间       0.0
到达结束时间       0.0
出发起始时间       0.0
出发结束时间       0.0
出发站          0.0
到达站          0.0
出发站编号        0.0
到达站编号        0.0
出发站车辆到站时间    0.0
到达站车辆到站时间    0.0
OST          0.0
OT           0.0
DT           0.0
DST          0.0
o1           0.0
o2           0.0
d1           0.0

...

时间划分完毕
空间划分完毕
栅格与上行站点匹配完毕
站点最近轨迹点匹配完毕
时间戳获取完毕
OD计算完毕
日期                                                  2405050214
到达时间         15:20-15:2515:30-15:3515:20-15:2515:35-15:4015...
D编号          wtubm1swtubm1swtubkcxwtubkcxwtubm1nwtubk0ewtub...
到达人数                                                      1649
出发时间         15:05-15:1015:00-15:0515:10-15:1514:55-15:0015...
O编号          wtubmc8wtubm9qwtubmchwtubr18wtubqcwwtubm0gwtub...
出发人数                                                       120
到达起始时间       15:2015:3015:2015:3515:3515:4015:4515:0015:301...
到达结束时间       15:2515:3515:2515:4015:4015:4515:5015:0515:351...
出发起始时间       15:0515:0015:1014:5515:1015:2015:1514:2515:151...
出发结束时间       15:1015:0515:1515:0015:1515:2515:2014:3015:201...
出发站          珍园（萃园）珍园（萃园）琼花观（银河电子城）市民中心（科技馆）市民中心（科技馆）廉政广场廉政...
到达站          廉政广场廉政广场东方医院（中国移动）东方医院（中国移动）市政府体育公园体育公园文昌东路曙光路...
出发站编号                                                 12248821
到达站编号                                                 12236034
出发

时间划分完毕
空间划分完毕
栅格与上行站点匹配完毕
站点最近轨迹点匹配完毕
时间戳获取完毕
OD计算完毕
日期                                                  3557049056
到达时间         11:40-11:4512:10-12:1511:55-12:0011:45-11:5011...
D编号          wtubm1swtubm1swtubm1swtubm9wwtubm1hwtubm1nwtub...
到达人数                                                      2976
出发时间         11:35-11:4011:55-12:0011:45-11:5011:35-11:4011...
O编号          wtubmchwtubm9rwtubm9qwtubr3gwtubr18wtubmchwtub...
出发人数                                                       183
到达起始时间       11:4012:1011:5511:4511:5512:0512:1011:5511:551...
到达结束时间       11:4512:1512:0011:5012:0012:1012:1512:0012:001...
出发起始时间       11:3511:5511:4511:3511:4011:5511:3011:4011:351...
出发结束时间       11:4012:0011:5011:4011:4512:0011:3511:4511:401...
出发站          琼花观（银河电子城）珍园（萃园）珍园（萃园）沙联市民中心（科技馆）琼花观（银河电子城）解放桥...
到达站          廉政广场廉政广场廉政广场珍园（萃园）廉政广场市政府市政府市政府东方医院（中国移动）东方医院（...
出发站编号                                                 18028933
到达站编号                                                 17961963
出发

时间划分完毕
空间划分完毕
栅格与下行站点匹配完毕
站点最近轨迹点匹配完毕
时间戳获取完毕
OD计算完毕
日期                                                  2627365780
到达时间         09:55-10:0009:55-10:0010:30-10:3510:15-10:2010...
D编号          wtubm9wwtubm9wwtubm0fwtubm0fwtubm1hwtubm1hwtub...
到达人数                                                      1719
出发时间         09:40-09:4509:40-09:4509:45-09:5009:40-09:4509...
O编号          wtubq3ewtubq9ewtubq17wtubr18wtubq33wtv02enwtub...
出发人数                                                       137
到达起始时间       09:5509:5510:3010:1510:0510:1510:2010:1010:151...
到达结束时间       10:0010:0010:3510:2010:1010:2010:2510:1510:201...
出发起始时间       09:4009:4009:4509:4009:5009:4509:5510:0010:000...
出发结束时间       09:4509:4509:5009:4509:5509:5010:0010:0510:050...
出发站          沙北三村鸿泰家园解放桥（旅游集散中心）市民中心（科技馆）扬州仁爱医院仙女庙大桥东文昌大桥文昌...
到达站          珍园（萃园）珍园（萃园）廉政广场廉政广场廉政广场廉政广场东方医院（中国移动）珍园（萃园）珍园...
出发站编号                                                 13398323
到达站编号                                                 13342652
出发

时间划分完毕
空间划分完毕
栅格与下行站点匹配完毕
站点最近轨迹点匹配完毕
时间戳获取完毕
OD计算完毕
日期                                                  2445471226
到达时间         13:50-13:5514:05-14:1014:10-14:1514:05-14:1014...
D编号          wtubm9wwtubq3ewtubqdnwtubqdnwtubq18wtv0250wtv0...
到达人数                                                      1755
出发时间         13:30-13:3513:50-13:5513:55-14:0013:35-13:4013...
O编号          wtubkc2wtubm1qwtubmc8wtubkc7wtubk27wtubm3fwtub...
出发人数                                                       124
到达起始时间       13:5014:0514:1014:0514:0014:0514:0514:2013:451...
到达结束时间       13:5514:1014:1514:1014:0514:1014:1014:2513:501...
出发起始时间       13:3013:5013:5513:3513:3013:3514:0014:0013:401...
出发结束时间       13:3513:5514:0013:4013:3513:4014:0514:0513:451...
出发站          来鹤台广场市政府珍园（萃园）翠岗小区双博馆人民大厦（中医院）市民中心（科技馆）琼花观（银河电...
到达站          珍园（萃园）沙北三村文昌大桥文昌大桥琼花观（银河电子城）文昌东路曙光路文昌东路曙光路文昌东路...
出发站编号                                                 12397918
到达站编号                                                 12412304
出发

ValueError: time data '1 day, 0:14:58' does not match format '%H:%M:%S'